# Exception Handling Mastery

**Level 2: Intermediate - Professional Error Management**

**Master advanced exception handling for production-quality Java applications**

---

## Custom Exceptions

**Domain-specific exceptions for meaningful error communication**

In [ ]:
// Custom exceptions communicate domain-specific errors
public class CustomExceptionsDemo {

    // Base custom exception
    static class BankException extends Exception {
        public BankException(String message) {
            super(message);
        }

        public BankException(String message, Throwable cause) {
            super(message, cause);
        }
    }

    // Specific business exceptions
    static class InsufficientFundsException extends BankException {
        private double required;
        private double available;

        public InsufficientFundsException(double required, double available) {
            super("Insufficient funds. Required: $" + required + ", Available: $" + available);
            this.required = required;
            this.available = available;
        }

        public double getShortfall() {
            return required - available;
        }
    }

    static class InvalidAccountException extends BankException {
        private String accountId;

        public InvalidAccountException(String accountId) {
            super("Invalid account: " + accountId);
            this.accountId = accountId;
        }

        public String getAccountId() {
            return accountId;
        }
    }

    static class BankService {
        private double balance = 1000.0;

        public void withdraw(double amount) throws InsufficientFundsException {
            if (amount <= 0) {
                throw new BankException("Withdrawal amount must be positive: " + amount);
            }

            if (amount > balance) {
                throw new InsufficientFundsException(amount, balance);
            }

            balance -= amount;
            System.out.println("Withdrawal successful: $" + amount + ". New balance: $" + balance);
        }

        public void transfer(String accountId, double amount) throws BankException {
            validateAccount(accountId);

            try {
                withdraw(amount);
                System.out.println("Transfer to " + accountId + " completed");
            } catch (InsufficientFundsException e) {
                throw new BankException("Transfer failed: " + e.getMessage(), e);
            }
        }

        private void validateAccount(String accountId) throws InvalidAccountException {
            if (accountId == null || accountId.isEmpty()) {
                throw new InvalidAccountException(accountId);
            }

            if (!accountId.matches("ACC\\d{6}")) { // Must be ACC followed by 6 digits
                throw new InvalidAccountException(accountId);
            }
        }

        public double getBalance() {
            return balance;
        }
    }

    public static void demonstrateCustomExceptions() {
        System.out.println("=== CUSTOM EXCEPTIONS FOR DOMAIN LOGIC ===\n");

        BankService bank = new BankService();

        System.out.println("Initial balance: $" + bank.getBalance() + "\n");

        // Test various scenarios
        testOperation("Valid withdrawal", () -> bank.withdraw(200));
        testOperation("Invalid amount", () -> bank.withdraw(-50));
        testOperation("Insufficient funds", () -> bank.withdraw(1000));

        testTransfer("Valid transfer", "ACC123456", 150);
        testTransfer("Invalid account", "INVALID", 100);
        testTransfer("Transfer over balance", "ACC123456", 1000);
    }

    private static void testOperation(String description, Runnable operation) {
        System.out.println("Testing: " + description);
        try {
            operation.run();
        } catch (BankException e) {
            System.out.println("❌ " + e.getClass().getSimpleName() + ": " + e.getMessage());
            if (e instanceof InsufficientFundsException) {
                InsufficientFundsException ife = (InsufficientFundsException) e;
                System.out.println("   Shortfall: $" + ife.getShortfall());
            }
        } catch (Exception e) {
            System.out.println("❌ Unexpected error: " + e.getMessage());
        }
        System.out.println();
    }

    private static void testTransfer(String description, String accountId, double amount) {
        System.out.println("Testing: " + description);
        try {
            BankService bank = new BankService();
            bank.transfer(accountId, amount);
        } catch (BankException e) {
            System.out.println("❌ " + e.getClass().getSimpleName() + ": " + e.getMessage());
        } catch (Exception e) {
            System.out.println("❌ Unexpected error: " + e.getMessage());
        }
        System.out.println();
    }

    public static void main(String[] args) {
        demonstrateCustomExceptions();

        System.out.println("🎯 CUSTOM EXCEPTIONS BENEFITS:");
        System.out.println("• Domain-specific error information");
        System.out.println("• Additional context through methods");
        System.out.println("• Clear exception hierarchy");
        System.out.println("• Type-safe exception handling");
        System.out.println("• Professional error communication");
    }
}


## Checked vs Unchecked Exceptions Mastery

**Strategic exception classification for proper handling**

In [ ]:
// Master the art of checked vs unchecked exception classification
import java.io.*;
import java.util.Scanner;

public class CheckedUncheckedMastery {

    // CHECKED EXCEPTIONS - Compiler forces handling
    public static void demonstrateCheckedExceptions() {
        System.out.println("=== CHECKED EXCEPTIONS (Compiler-Enforced) ===\n");

        System.out.println("Reading from file that might not exist...");

        try {
            // File operations throw IOException (checked)
            readFile("nonexistent.txt");
            System.out.println("File read successfully");

        } catch (IOException e) {
            System.out.println("❌ File operation failed: " + e.getMessage());
            handleIOException(e);
        }

        System.out.println("\nWITH CHECKED EXCEPTIONS:");
        System.out.println("• Compiler prevents unhandled errors");
        System.out.println("• Clear contract about possible failures");
        System.out.println("• IDE shows exactly what can fail");
        System.out.println("• Used for recoverable external errors");
    }

    private static void readFile(String filename) throws IOException {
        FileReader reader = new FileReader(filename);
        reader.close(); // This will succeed if file exists
    }

    private static void handleIOException(IOException e) {
        if (e instanceof FileNotFoundException) {
            System.out.println("SOLUTION: Check file path and permissions");
        } else {
            System.out.println("SOLUTION: Check file permissions and disk space");
        }
    }

    // UNCHECKED EXCEPTIONS - Programmer's choice
    public static void demonstrateUncheckedExceptions() {
        System.out.println("\n=== UNCHECKED EXCEPTIONS (Optional Handling) ===\n");

        System.out.println("Processing user data...");

        try {
            processUserData("", 25);
            System.out.println("Data processed successfully");

        } catch (IllegalArgumentException e) {
            System.out.println("❌ Validation failed: " + e.getMessage());
            System.out.println("This is programmer error, should be fixed not caught!");
        } catch (NullPointerException e) {
            System.out.println("❌ Null reference: " + e.getMessage());
            System.out.println("This indicates bug in code logic");
        }

        System.out.println("\nWITH UNCHECKED EXCEPTIONS:");
        System.out.println("• Compiler doesn't force handling");
        System.out.println("• Usually indicates programming errors");
        System.out.println("• Optional handling - fix the bug instead");
        System.out.println("• RuntimeException and subclasses");
    }

    private static void processUserData(String name, int age) {
        // Validate inputs (throws unchecked exceptions)
        if (name == null || name.trim().isEmpty()) {
            throw new IllegalArgumentException("Name cannot be null or empty");
        }

        if (age < 0 || age > 150) {
            throw new IllegalArgumentException("Age must be between 0 and 150: " + age);
        }

        // Simulate null pointer (would be programming bug)
        String upperName = name.toUpperCase(); // Could throw if name was null

        System.out.println("Processing: " + upperName + ", Age: " + age);
    }

    // CONVERTING BETWEEN CHECKED AND UNCHECKED
    public static void exceptionConversionStrategies() {
        System.out.println("\n=== EXCEPTION CONVERSION STRATEGIES ===\n");

        System.out.println("Strategy 1: Wrap checked in unchecked (for APIs that can't throw checked)");
        try {
            loadConfiguration();
        } catch (ConfigurationException e) { // Our unchecked exception
            System.out.println("❌ Configuration error: " + e.getMessage());
            if (e.getCause() instanceof IOException) {
                System.out.println("   Caused by I/O error - using default configuration");
            }
        }

        System.out.println("\nStrategy 2: Convert unchecked to checked (rare, but possible)");
        try {
            String result = safeArithmetic();
            System.out.println("Arithmetic result: " + result);
        } catch (ArithmeticException e) {
            throw new RuntimeException("Arithmetic failed", e); // Wrap to change type
        }
    }

    // Custom unchecked exception that wraps checked exception
    static class ConfigurationException extends RuntimeException {
        public ConfigurationException(String message, Throwable cause) {
            super(message, cause);
        }
    }

    private static void loadConfiguration() {
        try {
            // Some I/O operation that could fail
            FileReader reader = new FileReader("config.txt");
            reader.close();
            System.out.println("Configuration loaded successfully");
        } catch (IOException e) {
            // Convert checked IOException to unchecked ConfigurationException
            throw new ConfigurationException("Failed to load configuration", e);
        }
    }

    private static String safeArithmetic() {
        int result = 10 / 0; // Throws unchecked ArithmeticException
        return "Result: " + result;
    }

    // WHEN TO USE EACH TYPE
    public static void exceptionSelectionGuidelines() {
        System.out.println("\n=== EXCEPTION SELECTION GUIDELINES ===\n");

        System.out.println("USE CHECKED EXCEPTIONS:");
        System.out.println("• When caller must handle the error");
        System.out.println("• For recoverable external conditions");
        System.out.println("• When API users need to react to failure");
        System.out.println("• Examples: IOException, SQLException");

        System.out.println("\nUSE UNCHECKED EXCEPTIONS:");
        System.out.println("• For programming errors (bugs)");
        System.out.println("• When caller cannot reasonably recover");
        System.out.println("• For precondition violations");
        System.out.println("• Examples: NullPointerException, IllegalArgumentException");

        System.out.println("\nGENERAL RULE:");
        System.out.println("• Checked = External factors (file not found, network down)");
        System.out.println("• Unchecked = Internal factors (bad code, invalid parameters)");
    }

    public static void main(String[] args) {
        demonstrateCheckedExceptions();
        demonstrateUncheckedExceptions();
        exceptionConversionStrategies();
        exceptionSelectionGuidelines();

        System.out.println("\n🎯 EXCEPTION CLASSIFICATION MASTERY:");
        System.out.println("• Checked exceptions for expected, recoverable errors");
        System.out.println("• Unchecked for programming bugs and preconditions");
        System.out.println("• Strategic conversion when API design requires it");
        System.out.println("• Classification affects API usability and robustness");
    }
}


## Try-With-Resources Mastery

**Automatic resource management for bulletproof code**

In [ ]:
// Professional resource management with try-with-resources
import java.io.*;
import java.sql.*; // For database example
import java.util.Scanner;

public class TryWithResourcesMastery {

    // BASIC AUTO-CLOSABLE RESOURCE
    static class Resource implements AutoCloseable {
        private String name;
        private boolean closed = false;

        public Resource(String name) {
            this.name = name;
            System.out.println(name + " opened/acquired");
        }

        public void use() {
            if (closed) throw new IllegalStateException(name + " is closed");
            System.out.println(name + " is being used...");
        }

        @Override
        public void close() {
            if (!closed) {
                closed = true;
                System.out.println(name + " closed/released");
            }
        }
    }

    public static void demonstrateBasicTryWithResources() {
        System.out.println("=== BASIC TRY-WITH-RESOURCES ===\n");

        // Single resource
        try (Resource res = new Resource("File Handle")) {
            res.use();
            System.out.println("Resource work completed successfully");
            // close() called automatically here
        } catch (Exception e) {
            System.out.println("❌ Error: " + e.getMessage());
        }

        System.out.println("Resource automatically closed even with exception\n");
    }

    public static void demonstrateMultipleResources() {
        System.out.println("=== MULTIPLE RESOURCES MANAGEMENT ===\n");

        // Multiple resources - closed in reverse order
        try (
            Resource db = new Resource("Database Connection");
            Resource file = new Resource("Log File");
            Resource cache = new Resource("Memory Cache")
        ) {
            db.use();
            file.use();
            cache.use();

            System.out.println("All resources used successfully");
            // Resources closed in reverse order: cache, file, db

        } catch (Exception e) {
            System.out.println("❌ Error during multi-resource operation: " + e.getMessage());
        }

        System.out.println("All resources automatically cleaned up\n");
    }

    // REAL FILE I/O WITH TRY-WITH-RESOURCES
    public static void demonstrateFileIO() {
        System.out.println("=== PROFESSIONAL FILE I/O ===\n");

        // Write configuration file
        try (PrintWriter writer = new PrintWriter(new FileWriter("app.config"))) {
            writer.println("app.name=MyApp");
            writer.println("app.version=2.0");
            writer.println("database.url=jdbc:sqlite:db.sqlite");
            System.out.println("✅ Configuration written successfully");

        } catch (IOException e) {
            System.out.println("❌ Failed to write config: " + e.getMessage());
            return;
        }

        // Read configuration file
        try (Scanner scanner = new Scanner(new File("app.config"))) {
            System.out.println("Reading configuration:");
            while (scanner.hasNextLine()) {
                System.out.println("  " + scanner.nextLine());
            }
            System.out.println("✅ Configuration read successfully");

        } catch (FileNotFoundException e) {
            System.out.println("❌ Config file not found: " + e.getMessage());
        }

        System.out.println("\nFile handles automatically closed - no leaks!\n");
    }

    // COMPLEX RESOURCE SCENARIOS
    public static void demonstrateComplexResources() {
        System.out.println("=== COMPLEX RESOURCE MANAGEMENT ===\n");

        // Resources that depend on each other
        try (
            Resource network = new Resource("Network Connection");
            Resource encryption = new Resource("Encryption Context")
        ) {
            // Use resources in correct order
            encryption.use();  // Need encryption first
            network.use();     // Then network

            System.out.println("Secure communication established");

            // Simulate potential error
            if (Math.random() > 0.7) {
                throw new RuntimeException("Communication failure");
            }

            System.out.println("Data transmitted successfully");

        } catch (Exception e) {
            System.out.println("❌ Communication failed: " + e.getMessage());
        }

        System.out.println("Resources cleaned up in correct order (reverse allocation)\n");
    }

    // SUPPRESSED EXCEPTIONS
    public static void demonstrateSuppressedExceptions() {
        System.out.println("=== SUPPRESSED EXCEPTIONS HANDLING ===\n");

        // Custom AutoCloseable that can fail on close
        class FailingResource implements AutoCloseable {
            private String name;

            public FailingResource(String name) {
                this.name = name;
                System.out.println(name + " acquired");
            }

            public void doWork() throws Exception {
                throw new Exception("Work failed in " + name);
            }

            @Override
            public void close() throws Exception {
                throw new Exception("Failed to close " + name);
            }
        }

        try (FailingResource res = new FailingResource("Database")) {
            res.doWork(); // This throws exception

        } catch (Exception e) {
            System.out.println("Primary exception: " + e.getMessage());

            // Check for suppressed exceptions from close() failures
            Throwable[] suppressed = e.getSuppressed();
            if (suppressed.length > 0) {
                System.out.println("Suppressed close() exceptions:");
                for (Throwable t : suppressed) {
                    System.out.println("  " + t.getMessage());
                }
            }
        }

        System.out.println("\nPrimary exception takes precedence, suppressed logged\n");
    }

    public static void bestPractices() {
        System.out.println("=== TRY-WITH-RESOURCES BEST PRACTICES ===\n");

        System.out.println("✅ ALWAYS Use Try-With-Resources:");
        System.out.println("• For any AutoCloseable resource (File, Connection, Stream)");
        System.out.println("• Prevents resource leaks from unclosed resources");
        System.out.println("• Automatic cleanup even when exceptions occur");
        System.out.println("• Cleaner code than manual try-finally blocks");

        System.out.println("\n✅ Resource Declaration Rules:");
        System.out.println("• Resources declared inside parentheses after try");
        System.out.println("• Must implement AutoCloseable interface");
        System.out.println("• Closed in reverse order of declaration");
        System.out.println("• Each resource initialized separately (no sharing)");

        System.out.println("\n✅ Exception Handling:");
        System.out.println("• Primary exception propagated normally");
        System.out.println("• Close failures become suppressed exceptions");
        System.out.println("• Use getSuppressed() to inspect close failures");

        System.out.println("\n⚠️  CAUTION:");
        System.out.println("Don't nest try-with-resources (use multi-resource declaration)");
        System.out.println("Don't assign resources to variables outside try");
    }

    public static void main(String[] args) {
        demonstrateBasicTryWithResources();
        demonstrateMultipleResources();
        demonstrateFileIO();
        demonstrateComplexResources();
        demonstrateSuppressedExceptions();
        bestPractices();

        System.out.println("\n🎯 TRY-WITH-RESOURCES MASTERY:");
        System.out.println("Automatic resource management");
        System.out.println("Exception-safe cleanup"); 
        System.out.println("Clean, maintainable code");
        System.out.println("Zero resource leaks in production!");
    }
}
